# Deep Neural Network Model Design

**By: M. Alwi Sukra**

In [1]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
for physical_device in physical_devices: 
    tf.config.experimental.set_memory_growth(physical_device, True)

In [3]:
os.environ["TF_ENABLE_CUBLAS_TENSOR_OP_MATH_FP32"] = "1"
os.environ["TF_ENABLE_CUDNN_TENSOR_OP_MATH_FP32"] = "1"
os.environ["TF_ENABLE_CUDNN_RNN_TENSOR_OP_MATH_FP32"] = "1"
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION"] = "1"
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION_GRAPH_REWRITE"] = "1"
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION_LOSS_SCALING"] = "1"

## 1. DNN With Base Feature

In [4]:
DATA_PATH = "datasets/dnn-base"

# get X and y
X = np.load(f"{DATA_PATH}-X.npy")
y = np.load(f"{DATA_PATH}-y.npy")

model_name = "DNN_base-features"

# number of neurons on each layer
layers = [6,6,6]

# model structure
model = tf.keras.models.Sequential()
for neurons in layers:
    model.add(tf.keras.layers.Dense(neurons, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

# compile moodel
opt = tf.keras.optimizers.Adam()
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)
acc = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics = [acc])

# setup callbacks
checkpoint_filepath = f"saved-models/{model_name}" + "_ep-{epoch:02d}_valos-{val_loss:.4f}.hdf5"
log_filepath = f"csv-train-logs/{model_name}.csv"
# callbacks
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=0, mode='min', save_best_only=True)
csv_log = CSVLogger(log_filepath, separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
callbacks = [checkpoint, csv_log, reduce_lr, early_stop]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
np.save("datasets/X_test_"+model_name, X_test)
np.save("datasets/y_test_"+model_name, y_test)

In [6]:
MAX_EPOCHS = 100
model.fit(X_train, y_train, validation_split=0.1, epochs=MAX_EPOCHS, verbose=2, callbacks=callbacks)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1272/1272 - 3s - loss: 0.6833 - binary_accuracy: 0.5509 - val_loss: 0.6620 - val_binary_accuracy: 0.6091 - lr: 0.0010
Epoch 2/100
1272/1272 - 3s - loss: 0.6473 - binary_accuracy: 0.6209 - val_loss: 0.6508 - val_binary_accuracy: 0.6078 - lr: 0.0010
Epoch 3/100
1272/1272 - 3s - loss: 0.6399 - binary_accuracy: 0.6273 - val_loss: 0.6475 - val_binary_accuracy: 0.6160 - lr: 0.0010
Epoch 4/100
1272/1272 - 3s - loss: 0.6373 - binary_accuracy: 0.6283 - val_loss: 0.6453 - val_binary_accuracy: 0.6153 -

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  24        
_________________________________________________________________
dense_1 (Dense)              multiple                  42        
_________________________________________________________________
dense_2 (Dense)              multiple                  42        
_________________________________________________________________
dense_3 (Dense)              multiple                  7         
Total params: 115
Trainable params: 115
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.save("saved-models/"+model_name)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved-models/DNN_base-features\assets


## 2. DNN With Aggregate Feature

In [19]:
DATA_PATH = "datasets/dnn-agg"

# get X and y
X = np.load(f"{DATA_PATH}-X.npy")
y = np.load(f"{DATA_PATH}-y.npy")

model_name = "DNN_agg-features"

# number of neurons on each layer
layers = [6,6,6]

# model structure
model2 = tf.keras.models.Sequential()
for neurons in layers:
    model2.add(tf.keras.layers.Dense(neurons, activation=tf.nn.relu))
model2.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

# compile moodel
opt = tf.keras.optimizers.Adam()
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)
acc = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
model2.compile(optimizer=opt, loss="binary_crossentropy", metrics = [acc])

# setup callbacks
checkpoint_filepath = f"saved-models/{model_name}" + "_ep-{epoch:02d}_valos-{val_loss:.4f}.hdf5"
log_filepath = f"csv-train-logs/{model_name}.csv"
# callbacks
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=0, mode='min', save_best_only=True)
csv_log = CSVLogger(log_filepath, separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
callbacks = [checkpoint, csv_log, reduce_lr, early_stop]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
np.save("datasets/X_test_"+model_name, X_test)
np.save("datasets/y_test_"+model_name, y_test)

In [21]:
MAX_EPOCHS = 100
model2.fit(X_train, y_train, validation_split=0.1, epochs=MAX_EPOCHS, verbose=2, callbacks=callbacks)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1272/1272 - 3s - loss: 0.4856 - binary_accuracy: 0.7930 - val_loss: 0.3656 - val_binary_accuracy: 0.7997 - lr: 0.0010
Epoch 2/100
1272/1272 - 3s - loss: 0.3579 - binary_accuracy: 0.7945 - val_loss: 0.3501 - val_binary_accuracy: 0.8030 - lr: 0.0010
Epoch 3/100
1272/1272 - 3s - loss: 0.3512 - binary_accuracy: 0.8006 - val_loss: 0.3467 - val_binary_accuracy: 0.8194 - lr: 0.0010
Epoch 4/100
1272/1272 - 3s - loss: 0.3479 - binary_accuracy: 0.8060 - val_loss: 0.3429 - val_binary_accuracy: 0.8037 -

In [12]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  24        
_________________________________________________________________
dense_5 (Dense)              multiple                  42        
_________________________________________________________________
dense_6 (Dense)              multiple                  42        
_________________________________________________________________
dense_7 (Dense)              multiple                  7         
Total params: 115
Trainable params: 115
Non-trainable params: 0
_________________________________________________________________


In [13]:
model2.save("saved-models/"+model_name)

INFO:tensorflow:Assets written to: saved-models/DNN_agg-features\assets


## 3. DNN With All Feature

In [14]:
DATA_PATH = "datasets/dnn-all"

# get X and y
X = np.load(f"{DATA_PATH}-X.npy")
y = np.load(f"{DATA_PATH}-y.npy")

model_name = "DNN_all-features"

# number of neurons on each layer
layers = [6,6,6]

# model structure
model3 = tf.keras.models.Sequential()
for neurons in layers:
    model3.add(tf.keras.layers.Dense(neurons, activation=tf.nn.relu))
model3.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

# compile moodel
opt = tf.keras.optimizers.Adam()
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)
acc = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
model3.compile(optimizer=opt, loss="binary_crossentropy", metrics = [acc])

# setup callbacks
checkpoint_filepath = f"saved-models/{model_name}" + "_ep-{epoch:02d}_valos-{val_loss:.4f}.hdf5"
log_filepath = f"csv-train-logs/{model_name}.csv"
# callbacks
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=0, mode='min', save_best_only=True)
csv_log = CSVLogger(log_filepath, separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
callbacks = [checkpoint, csv_log, reduce_lr, early_stop]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
np.save("datasets/X_test_"+model_name, X_test)
np.save("datasets/y_test_"+model_name, y_test)

In [16]:
MAX_EPOCHS = 100
model3.fit(X_train, y_train, validation_split=0.1, epochs=MAX_EPOCHS, verbose=2, callbacks=callbacks)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1272/1272 - 3s - loss: 0.5449 - binary_accuracy: 0.7519 - val_loss: 0.4555 - val_binary_accuracy: 0.8092 - lr: 0.0010
Epoch 2/100
1272/1272 - 3s - loss: 0.4234 - binary_accuracy: 0.8250 - val_loss: 0.3940 - val_binary_accuracy: 0.8333 - lr: 0.0010
Epoch 3/100
1272/1272 - 3s - loss: 0.3853 - binary_accuracy: 0.8335 - val_loss: 0.3730 - val_binary_accuracy: 0.8096 - lr: 0.0010
Epoch 4/100
1272/1272 - 3s - loss: 0.3667 - binary_accuracy: 0.8352 - val_loss: 0.3514 - val_binary_accuracy: 0.8587 -

In [17]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              multiple                  42        
_________________________________________________________________
dense_9 (Dense)              multiple                  42        
_________________________________________________________________
dense_10 (Dense)             multiple                  42        
_________________________________________________________________
dense_11 (Dense)             multiple                  7         
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


In [18]:
model3.save("saved-models/"+model_name)

INFO:tensorflow:Assets written to: saved-models/DNN_all-features\assets
